In [22]:
import sqlite3

                                                       # Create a connection to the SQLite database
conn = sqlite3.connect('library.db')
c = conn.cursor()

                                                   # Create the tables
c.execute('''CREATE TABLE IF NOT EXISTS Books
             (BookID INTEGER PRIMARY KEY, Title TEXT, Author TEXT, PublicationYear INTEGER)''')

c.execute('''CREATE TABLE IF NOT EXISTS Members
             (MemberID INTEGER PRIMARY KEY, Name TEXT, MembershipDate DATE)''')

c.execute('''CREATE TABLE IF NOT EXISTS Borrowing
             (BorrowingID INTEGER PRIMARY KEY, BookID INTEGER, MemberID INTEGER, BorrowDate DATE, ReturnDate DATE,
              FOREIGN KEY (BookID) REFERENCES Books(BookID),
              FOREIGN KEY (MemberID) REFERENCES Members(MemberID))''')

conn.commit()

                                                             # Function to add a new book
def add_book(title, author, publication_year):
    c.execute("INSERT INTO Books (Title, Author, PublicationYear) VALUES (?, ?, ?)", (title, author, publication_year))
    conn.commit()
    print(f"Added book: {title}")

                                                #  Function to add a new member
def add_member(name, membership_date):
    c.execute("INSERT INTO Members (Name, MembershipDate) VALUES (?, ?)", (name, membership_date))
    conn.commit()
    print(f"Added member: {name}")


def borrow_book(member_id, book_id, borrow_date):                   # Function to borrow a book
    c.execute("INSERT INTO Borrowing (BookID, MemberID, BorrowDate) VALUES (?, ?, ?)", (book_id, member_id, borrow_date))
    conn.commit()
    print(f"Book borrowed by member {member_id}")


add_book("The Great Cats", "Abdullah Alshehri", 1999)                # Example usage
add_member("Omar Done by : Abdullah Al-shehri,  Fares Al-Nemer,   Abdulaziz Al-Mudrfah", date(2022, 1, 1))
borrow_book(1, 1, date(2023, 6, 1))



conn.close()           # Close the database connection
 


Added book: The Great Cats
Added member: Omar Done by : Abdullah Al-shehri,  Fares Al-Nemer,   Abdulaziz Al-Mudrfah
Book borrowed by member 1
